# Caesar Cipher Implementation and Security Analysis
## Gabe DiMartino

This notebook implements a Caesar cipher with an extended alphabet (A-Z plus space), demonstrates all possible cipher texts for given messages, and proposes the best case scenario for a Caesar cipher with perfect secrecy.

## 1. Setup and Constants

In [13]:
# Define the alphabet including space character
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ "

# Create bidirectional mappings between characters and numbers
CHAR_TO_NUM = {char: i for i, char in enumerate(ALPHABET)}
NUM_TO_CHAR = list(ALPHABET)

# Display the mappings
print("Alphabet length:", len(ALPHABET))
print("Character to number mapping (first 5):", 
      {k: v for k, v in list(CHAR_TO_NUM.items())[:5]})
print("Space character maps to:", CHAR_TO_NUM[' '])

Alphabet length: 27
Character to number mapping (first 5): {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
Space character maps to: 26


## 2. Core Caesar Cipher Implementation

The Caesar cipher shifts each character by a fixed amount determined by the key character.

In [14]:
def caesar_cipher(message, key):
    shift = CHAR_TO_NUM[key]
    encrypted_message = ""
    
    for char in message:
        if char not in CHAR_TO_NUM:
            raise ValueError(f"Character '{char}' in message is not in the alphabet")
        
        char_num = CHAR_TO_NUM[char]
        encrypted_num = (char_num + shift) % 27
        encrypted_message += NUM_TO_CHAR[encrypted_num]
    
    return encrypted_message

In [15]:
test_message = "HELLO"
print(f"Original message: '{test_message}'")
print(f"Encrypted with key 'A' (shift 0): '{caesar_cipher(test_message, 'A')}'")
print(f"Encrypted with key 'D' (shift 3): '{caesar_cipher(test_message, 'D')}'")
print(f"Encrypted with key 'Z' (shift 25): '{caesar_cipher(test_message, 'Z')}'")

Original message: 'HELLO'
Encrypted with key 'A' (shift 0): 'HELLO'
Encrypted with key 'D' (shift 3): 'KHOOR'
Encrypted with key 'Z' (shift 25): 'FCJJM'


## 3. Finding Encryption Keys

This function determines which key (if any) would encrypt a given message to produce a target ciphertext. (To be used in part 3)

In [16]:
def find_key_for_encryption(message, target):
    if len(message) != len(target):
        return None
    
    # Calculate required shift from first character
    shift = (CHAR_TO_NUM[target[0]] - CHAR_TO_NUM[message[0]]) % 27
    
    # Verify shift works for all characters
    for i in range(len(message)):
        expected_shift = (CHAR_TO_NUM[target[i]] - CHAR_TO_NUM[message[i]]) % 27
        if expected_shift != shift:
            return None
    
    return NUM_TO_CHAR[shift]

In [17]:
# Example: Find key to encrypt "HELLO" to "KHOOR"
key = find_key_for_encryption("HELLO", "KHOOR")
print(f"Key to encrypt 'HELLO' to 'KHOOR': '{key}'")
print(f"Verification: {caesar_cipher('HELLO', key)}")

Key to encrypt 'HELLO' to 'KHOOR': 'D'
Verification: KHOOR


## 4. Generating All Possible Ciphertexts For A Singular Message

For a given message, this function can generate all 27 possible ciphertexts (one for each key in the alphabet).

In [18]:
def generate_all_ciphertexts(message):
    ciphertexts = {}
    for key in ALPHABET:
        ciphertext = caesar_cipher(message, key)
        ciphertexts[key] = ciphertext
    return ciphertexts

In [19]:
# Generate all ciphertexts for "GO ZAGS"
message = "GO ZAGS"
all_ciphertexts = generate_all_ciphertexts(message)

print(f"All ciphertexts for '{message}':")
print("=" * 50)
for i, (key, ciphertext) in enumerate(all_ciphertexts.items()):
    key_display = "SPACE" if key == " " else key
    shift = CHAR_TO_NUM[key]
    print(f"Key: {key_display:5} (shift={shift:2}) -> '{ciphertext}'")
    if i >= 5:
        print("... (21 more ciphertexts)")
        break

All ciphertexts for 'GO ZAGS':
Key: A     (shift= 0) -> 'GO ZAGS'
Key: B     (shift= 1) -> 'HPA BHT'
Key: C     (shift= 2) -> 'IQBACIU'
Key: D     (shift= 3) -> 'JRCBDJV'
Key: E     (shift= 4) -> 'KSDCEKW'
Key: F     (shift= 5) -> 'LTEDFLX'
... (21 more ciphertexts)


In [20]:
def write_ciphertexts_to_file(filename, message, ciphertexts):
    with open(filename, 'w') as f:
        f.write(f"All possible ciphertexts for message: '{message}'\n")
        f.write("=" * 60 + "\n\n")
        
        for key in ALPHABET:
            shift = CHAR_TO_NUM[key]
            key_display = "SPACE" if key == " " else key
            f.write(f"Key: {key_display:5} (shift={shift:2}) -> '{ciphertexts[key]}'\n")

## 5. Finding Common Ciphertexts

A set of messages is "semantically secure" if there exists a ciphertext that could be the encryption of any message in the set (under different keys). This demonstrates perfect secrecy, an adversary seeing the ciphertext cannot determine which original message was encrypted. (This will be important for Part 3)

In [21]:
def find_common_ciphertext(messages):
    # Check all messages have same length
    message_length = len(messages[0])
    if not all(len(msg) == message_length for msg in messages):
        print("Error: All messages must have the same length")
        return None
    
    # Try each possible target ciphertext
    for test_key in ALPHABET:
        target = caesar_cipher(messages[0], test_key)
        
        key_mappings = {}
        all_can_reach = True
        
        # Check if all messages can reach this target
        for msg in messages:
            key = find_key_for_encryption(msg, target)
            if key is None:
                all_can_reach = False
                break
            else:
                key_mappings[msg] = key
        
        if all_can_reach:
            return (target, key_mappings)
    
    return None

In [22]:
def demonstrate_secure_message_set(messages, target_ciphertext, output_file):
    with open(output_file, 'w') as f:
        f.write("SEMANTICALLY INTERESTING AND SECURE \n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Message Space: {messages}\n")
        f.write(f"Target Ciphertext: '{target_ciphertext}'\n")
        f.write("-" * 60 + "\n\n")
        
        valid_mappings = []
        
        for msg in messages:
            key = find_key_for_encryption(msg, target_ciphertext)
            if key:
                encrypted = caesar_cipher(msg, key)
                shift = CHAR_TO_NUM[key]
                key_display = "SPACE" if key == " " else key
                
                f.write(f"Message: '{msg}'\n")
                f.write(f"  Key: '{key_display}' (shift = {shift})\n")
                f.write(f"  Encrypts to: '{encrypted}'\n")
                f.write(f"  Verification: '{encrypted}' == '{target_ciphertext}' -> {encrypted == target_ciphertext}\n\n")
                
                valid_mappings.append((msg, key))

## 6. Part 2: Generating All Possible Ciphertexts

In [ ]:
print("=" * 70)
print("PART 2: GENERATING ALL POSSIBLE CIPHERTEXTS")
print("=" * 70)
print()

# Messages to encrypt
messages_to_encrypt = ["GO ZAGS", "MESSAGE", "KRYPTOS"]

for msg in messages_to_encrypt:
    print(f"\nMessage: '{msg}'")
    print("-" * 40)
    
    ciphertexts = generate_all_ciphertexts(msg)
    
    filename = f"ciphertexts_{msg.replace(' ', '_')}.txt"
    write_ciphertexts_to_file(filename, msg, ciphertexts)
    print(f"✓ Written all 27 ciphertexts to '{filename}'")
    
    print("\nSample ciphertexts:")
    sample_keys = ['A', 'D', 'K', 'Z', ' ']
    for key in sample_keys:
        key_display = "SPACE" if key == " " else key
        print(f"  Key {key_display}: '{ciphertexts[key]}'")

PART 2: GENERATING ALL POSSIBLE CIPHERTEXTS


Message: 'GO ZAGS'
----------------------------------------
✓ Written all 27 ciphertexts to 'ciphertexts_GO_ZAGS.txt'

Sample ciphertexts:
  Key A: 'GO ZAGS'
  Key D: 'JRCBDJV'
  Key K: 'QYJIKQB'
  Key Z: 'EMYXZEQ'
  Key SPACE: 'FNZY FR'

Message: 'MESSAGE'
----------------------------------------
✓ Written all 27 ciphertexts to 'ciphertexts_MESSAGE.txt'

Sample ciphertexts:
  Key A: 'MESSAGE'
  Key D: 'PHVVDJH'
  Key K: 'WOBBKQO'
  Key Z: 'KCQQZEC'
  Key SPACE: 'LDRR FD'

Message: 'KRYPTOS'
----------------------------------------
✓ Written all 27 ciphertexts to 'ciphertexts_KRYPTOS.txt'

Sample ciphertexts:
  Key A: 'KRYPTOS'
  Key D: 'NUASWRV'
  Key K: 'UAHZCYB'
  Key Z: 'IPWNRMQ'
  Key SPACE: 'JQXOSNR'


## 7. Part 3: Secure Message Set

For this section it is important to note that only very specific words in the English Language can be used due to the spacing between letters. In order to satisfy the constraints of a Caesar cipher, the steps between each letter need to be the same for each message in the message space. Using a python helper function I queried the dictionary and went through every possible cipher text for every word in the English Language. Out of 370,000+ words only 37 sets of 3+ existed.

In [24]:
print("\n" + "=" * 70)
print("PART 3: SECURE MESSAGE SET")
print("=" * 70)
print()

secure_messages = ["CHEER", "DIFFS", "JOLLY"]

result = find_common_ciphertext(secure_messages)

if result:
    target, key_mappings = result
    print(f"Messages: {secure_messages}")
    print(f"Common Ciphertext Found: '{target}'")
    print("\nKey Mappings:")
    for msg, key in key_mappings.items():
        shift = CHAR_TO_NUM[key]
        key_display = "SPACE" if key == " " else key
        encrypted = caesar_cipher(msg, key)
        print(f"  '{msg}' + Key '{key_display}' (shift {shift:2}) -> '{encrypted}'")
    
    demonstrate_secure_message_set(secure_messages, target, "secure_message_demonstration.txt")
else:
    print("No common ciphertext could be found for the given messages.")


PART 3: SECURE MESSAGE SET

Messages: ['CHEER', 'DIFFS', 'JOLLY']
Common Ciphertext Found: 'CHEER'

Key Mappings:
  'CHEER' + Key 'A' (shift  0) -> 'CHEER'
  'DIFFS' + Key 'SPACE' (shift 26) -> 'CHEER'
  'JOLLY' + Key 'U' (shift 20) -> 'CHEER'


## 8. Security Analysis

The demonstration above shows a fundamental concept in cryptography: **perfect secrecy**. When an adversary intercepts the ciphertext 'CHEER', they cannot determine which of the three messages was originally sent. Returning to the probability statement from class:
Pr(Message = M | Cipher = C) = Pr(Message = M) is satisfied. This is because the cipher text gives no context of the message. Each message is equally likely if the key was chosen uniformly at random. 

### Key Observations:
- All three different messages ('CHEER', 'DIFFS', 'JOLLY') can produce the same ciphertext 'CHEER'
- Each uses a different key (A, SPACE, and U respectively)
- The Caesar cipher achieves perfect secrecy when the key space equals the message space size
- This is now a key distribution problem ensuring all parties with a need to know get the correct key to decipher the correct message.